In [1]:
from meta_transformer import torch_utils, module_path, on_cluster
import os
from time import time
from dataclasses import dataclass
import torch
from meta_transformer.data import split_data
import numpy as np

/home/lauro/.virtualenvs/meta-models/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
@dataclass
class Args:
    ndata: int = 23_710
    dataset: str = 'mnist'
    chunk_size: int = 256

args = Args()

args = Args(
    ndata=1000,
    dataset='cifar10',
)

if args.dataset == 'mnist':
    architecture = torch_utils.CNNSmall()  # for MNIST
elif args.dataset == 'cifar10':
    architecture = torch_utils.CNNMedium()  # for CIFAR-10

In [3]:
print(f'Initial memory allocated: {torch.cuda.memory_allocated() / (1024 ** 2)} MB')
print(f'Initial memory reserved: {torch.cuda.memory_reserved() / (1024 ** 2)} MB')

Initial memory allocated: 0.0 MB
Initial memory reserved: 0.0 MB


In [4]:
%%prun -s cumtime -l 30 -T 01_loading_data.txt

if not on_cluster:
    dpath = os.path.join(module_path, "data/david_backdoors")  # local
    # use for testing with small dataset sizes (only works if rds storage is mounted):
    # dpath = os.path.join(module_path, "/home/lauro/rds/model-zoo/")
else:
    dpath = "/rds/user/lsl38/rds-dsk-lab-eWkDxBhxBrQ/model-zoo/"  

model_dataset_paths = {
    "mnist": "mnist-cnns",
    "cifar10": "cifar10",
    "svhn": "svhn",
}

model_dataset_paths = {
    k: os.path.join(dpath, v) for k, v in model_dataset_paths.items()
}

inputs_dirnames = {
    "mnist": "poison_noL1reg",
    #"mnist": "poison",
    #"cifar10": "poison_noL1",
    "cifar10": "poison_simple",
    #"cifar10": "poison_easy6_alpha_50",
    "svhn": "poison_noL1",
}


print("Loading data...")
s = time()
inputs, targets, get_pytorch_model = torch_utils.load_input_and_target_weights(
    model=architecture,
    num_models=args.ndata,
    data_dir=model_dataset_paths[args.dataset],
    inputs_dirname=inputs_dirnames[args.dataset],
    targets_dirname="clean",
)
print("Data loading and processing took", round(time() - s), "seconds.")


#inputs_dir = os.path.join(model_dataset_paths[args.dataset], inputs_dirnames[args.dataset])
#targets_dir = os.path.join(model_dataset_paths[args.dataset], "clean")
#
#print("Loading data...")
#s = time()
#inputs, targets = torch_utils.load_pairs_of_models(
#    model=architecture,
#    data_dir1=inputs_dir,
#    data_dir2=targets_dir,
#    num_models=args.ndata,
#)
#print("Data loading and processing took", round(time() - s), "seconds.")

Loading data...
Could not find /home/lauro/projects/meta-models/meta-transformer/data/david_backdoors/cifar10/clean/clean_0179_0027.pth.
Could not find /home/lauro/projects/meta-models/meta-transformer/data/david_backdoors/cifar10/clean/clean_0143_0055.pth.
Could not find /home/lauro/projects/meta-models/meta-transformer/data/david_backdoors/cifar10/clean/clean_0263_0050.pth.
Could not find /home/lauro/projects/meta-models/meta-transformer/data/david_backdoors/cifar10/clean/clean_0066_0069.pth.
Could not find /home/lauro/projects/meta-models/meta-transformer/data/david_backdoors/cifar10/clean/clean_0143_0079.pth.
Could not find /home/lauro/projects/meta-models/meta-transformer/data/david_backdoors/cifar10/clean/clean_0095_0017.pth.
Could not find /home/lauro/projects/meta-models/meta-transformer/data/david_backdoors/cifar10/clean/clean_0280_0070.pth.
Could not find /home/lauro/projects/meta-models/meta-transformer/data/david_backdoors/cifar10/clean/clean_0073_0075.pth.
Could not find /

KeyboardInterrupt: 

In [ ]:
print(f'Current memory allocated: {torch.cuda.memory_allocated() / (1024 ** 2)} MB')
print(f'Current memory reserved: {torch.cuda.memory_reserved() / (1024 ** 2)} MB')

Current memory allocated: 0.0 MB
Current memory reserved: 0.0 MB


In [ ]:

(train_inputs, train_targets, 
    val_inputs, val_targets) = split_data(inputs, targets, 0.2)

In [ ]:
from meta_transformer import preprocessing
weights_std = 0.05

init_batch = {
    "input": train_inputs[:2],
    "target": train_targets[:2],
}

init_batch = preprocessing.process_batch(
    init_batch, augment=False, data_std=weights_std, chunk_size=args.chunk_size)

In [ ]:
print(f'Current memory allocated: {torch.cuda.memory_allocated() / (1024 ** 2)} MB')
print(f'Current memory reserved: {torch.cuda.memory_reserved() / (1024 ** 2)} MB')

Current memory allocated: 0.0 MB
Current memory reserved: 0.0 MB


In [ ]:
type(inputs)

numpy.ndarray

In [ ]:
type(inputs[0])

dict

In [ ]:
inputs[0].keys()

dict_keys(['Conv2d_0', 'Conv2d_1', 'Linear_2', 'Linear_3'])

In [ ]:
type(inputs[0]['Conv2d_0']['w'])

numpy.ndarray